סטודנטים:
גנא מ\פאיז נ\מג'ד א

3. agentic RAG


In [42]:
# =========================
# Cell 1: Imports + .env setup
# =========================
import os
import re
import requests
from pathlib import Path
from typing import Dict, Any, Literal
from dotenv import load_dotenv

# Load environment variables from .env in project root
env_path = Path.cwd() / ".env"
loaded = load_dotenv(dotenv_path=env_path, override=True)

groq_key = os.getenv("GROQ_API_KEY")
assert groq_key, (
    "❌ GROQ_API_KEY not found. "
    "Make sure .env exists in project root and contains GROQ_API_KEY=..."
)

print("✓ .env loaded:", loaded)
print(f"✓ GROQ_API_KEY loaded (length: {len(groq_key)})")
print("✓ Environment setup complete")


✓ .env loaded: True
✓ GROQ_API_KEY loaded (length: 56)
✓ Environment setup complete


In [43]:
# =========================
# Cell 2: Load docs + chunking
# =========================
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

DOC_PATH = Path("data") / "docs.txt"
assert DOC_PATH.exists(), f"❌ Missing file: {DOC_PATH.resolve()}"

docs = TextLoader(str(DOC_PATH), encoding="utf-8").load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=120,
    separators=["\n\n", "\n", ". ", " ", ""],
)

chunks = splitter.split_documents(docs)

print("✓ Loaded docs:", len(docs))
print("✓ Chunks:", len(chunks))
print("✓ Example chunk chars:", len(chunks[0].page_content))
print("✓ Preview:\n", chunks[0].page_content[:250])


✓ Loaded docs: 1
✓ Chunks: 5
✓ Example chunk chars: 749
✓ Preview:
 Information Retrieval (IR) Notes

Information Retrieval deals with finding relevant documents for a given user query.
The goal of IR systems is to rank documents by relevance rather than return exact matches.

An inverted index is the core data struc


In [44]:
# =========================
# Cell 3: Build / load FAISS + retriever
# =========================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

INDEX_DIR = Path("vectorstore_faiss")

# Local embeddings model (downloads once)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Load existing index if found, else build new
if INDEX_DIR.exists() and (INDEX_DIR / "index.faiss").exists():
    print("Loading existing FAISS index...")
    vectorstore = FAISS.load_local(str(INDEX_DIR), embeddings, allow_dangerous_deserialization=True)
    print("✓ Loaded existing FAISS index.")
else:
    print("Building new FAISS index...")
    INDEX_DIR.mkdir(exist_ok=True)
    vectorstore = FAISS.from_documents(chunks, embeddings)
    vectorstore.save_local(str(INDEX_DIR))
    print("✓ Built + saved new FAISS index to:", INDEX_DIR.resolve())

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# quick retrieval test
hits = retriever.invoke("What is TF-IDF?")
print("✓ Retriever working. Retrieved:", len(hits), "chunks")
print("✓ First hit preview:\n", hits[0].page_content[:200])


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Loading existing FAISS index...
✓ Loaded existing FAISS index.
✓ Retriever working. Retrieved: 3 chunks
✓ First hit preview:
 Stemming reduces words to their root form, for example "retrieval" to "retriev".
Lemmatization reduces words to their dictionary form and is linguistically informed.

Term Frequency (TF) measures how 


In [45]:
# =========================
# Cell 4: RAG Answer function (Groq)
# =========================
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

def rag_answer(question: str, k: int = 3) -> Dict[str, Any]:
    """
    RAG = retrieve top-k chunks from FAISS, then ask Groq to answer using ONLY those chunks.
    Returns: {"answer": str, "sources": [str, ...]}
    """
    # 1) Retrieve
    top_docs = retriever.invoke(question)[:k]

    # 2) Format sources
    context_blocks = []
    for i, d in enumerate(top_docs, 1):
        context_blocks.append(f"[Source {i}]\n{d.page_content.strip()}")

    context = "\n\n".join(context_blocks)

    # 3) Grounded prompt
    system = (
        "You are a RAG assistant for an Information Retrieval homework. "
        "Answer using ONLY the provided sources. "
        "If the sources do not contain the answer, say you don't have enough information. "
        "Cite sources like [Source 1], [Source 2]."
    )

    user = f"""Question: {question}

Sources:
{context}

Write a concise answer with citations."""

    # 4) Generate
    resp = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=0.2,
    )

    return {"answer": resp.choices[0].message.content, "sources": context_blocks}

# Quick test
print("Testing RAG function...")
test_result = rag_answer("What is TF-IDF?")
print("✓ RAG function working")
print("Answer preview:\n", test_result["answer"][:200], "...")


Testing RAG function...
✓ RAG function working
Answer preview:
 TF-IDF (Term Frequency-Inverse Document Frequency) is a weighting scheme used in Information Retrieval to measure the importance of a term in a document. It is computed as the product of Term Frequenc ...


In [47]:
# =========================
# Cell 5: Weather API (Open-Meteo) + Geocoding
# =========================
def get_weather_open_meteo(city: str) -> Dict[str, Any]:
    """
    API #1: Open-Meteo
    1) Geocoding API: city -> lat/lon
    2) Forecast API: lat/lon -> current weather
    No API key required.
    """
    try:
        # 1) Geocode city -> lat/lon
        geo_url = "https://geocoding-api.open-meteo.com/v1/search"
        geo = requests.get(
            geo_url,
            params={"name": city, "count": 1, "language": "en", "format": "json"},
            timeout=10
        )
        geo.raise_for_status()
        geo_data = geo.json()

        if not geo_data.get("results"):
            return {"error": f"City not found: {city}"}

        r0 = geo_data["results"][0]
        lat, lon = r0["latitude"], r0["longitude"]

        # 2) Current weather
        forecast_url = "https://api.open-meteo.com/v1/forecast"
        fc = requests.get(
            forecast_url,
            params={
                "latitude": lat,
                "longitude": lon,
                "current": "temperature_2m,wind_speed_10m",
                "timezone": "auto",
            },
            timeout=10
        )
        fc.raise_for_status()
        fc_data = fc.json()

        cur = fc_data.get("current", {})
        return {
            "city": r0.get("name"),
            "country": r0.get("country"),
            "lat": lat,
            "lon": lon,
            "temperature_c": cur.get("temperature_2m"),
            "wind_kmh": cur.get("wind_speed_10m"),
            "time": cur.get("time"),
            "source": "open-meteo"
        }

    except Exception as e:
        return {"error": f"Weather API error: {e}"}

# Quick test
w = get_weather_open_meteo("Haifa")
print("✓ Weather API test:", w if "error" in w else f"{w['city']} {w['temperature_c']}°C, wind {w['wind_kmh']} km/h")


✓ Weather API test: Haifa 18.1°C, wind 10.0 km/h


In [48]:
# =========================
# Cell 6: Crypto API (CoinGecko) - BONUS
# =========================
def get_crypto_price(symbol: str = "bitcoin") -> Dict[str, Any]:
    """
    API #2 (BONUS): CoinGecko simple price endpoint.
    No API key required.
    """
    try:
        url = "https://api.coingecko.com/api/v3/simple/price"
        params = {"ids": symbol.lower(), "vs_currencies": "usd", "include_24hr_change": "true"}
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        if symbol.lower() not in data:
            return {"error": f"Crypto not found: {symbol}"}

        coin = data[symbol.lower()]
        return {
            "symbol": symbol.lower(),
            "price_usd": coin.get("usd"),
            "change_24h": coin.get("usd_24h_change"),
            "source": "coingecko"
        }

    except Exception as e:
        return {"error": f"Crypto API error: {e}"}

# Quick test
c = get_crypto_price("bitcoin")
print("✓ Crypto API test:", c if "error" in c else f"bitcoin ${c['price_usd']}, 24h {c['change_24h']:.2f}%")


✓ Crypto API test: bitcoin $77858, 24h -7.59%


In [49]:
# =========================
# Cell 7: Extract helpers (city + crypto)
# =========================
def extract_city(user_query: str) -> str:
    """
    Extracts a city from patterns like:
    'weather in Tel Aviv', 'forecast for Paris'
    Fallback: Haifa
    """
    m = re.search(r"\b(in|at|for)\s+([A-Za-z\s\-]+)", user_query.strip(), re.IGNORECASE)
    if m:
        return m.group(2).strip()

    common = ["haifa", "tel aviv", "jerusalem", "london", "new york", "paris", "tokyo"]
    q = user_query.lower()
    for city in common:
        if city in q:
            return city.title()

    return "Haifa"

def extract_crypto(user_query: str) -> str:
    """
    Maps common symbols/names to CoinGecko ids.
    Fallback: bitcoin
    """
    q = user_query.lower()
    crypto_map = {
        "bitcoin": "bitcoin", "btc": "bitcoin",
        "ethereum": "ethereum", "eth": "ethereum",
        "solana": "solana", "sol": "solana",
        "cardano": "cardano", "ada": "cardano",
        "dogecoin": "dogecoin", "doge": "dogecoin",
    }
    for key, coin_id in crypto_map.items():
        if key in q:
            return coin_id
    return "bitcoin"

print("✓ Extractors ready:",
      extract_city("weather in Tel Aviv"), "|",
      extract_crypto("price of ETH"))


✓ Extractors ready: Tel Aviv | ethereum


In [50]:
# =========================
# Cell 8: Routing logic (agent chooses tool)
# =========================
def route_task(user_query: str) -> Literal["weather", "crypto", "rag"]:
    q = user_query.lower()

    crypto_keywords = ["crypto", "cryptocurrency", "bitcoin", "ethereum", "price of", "btc", "eth", "coin", "solana", "doge", "cardano"]
    if any(k in q for k in crypto_keywords):
        return "crypto"

    weather_keywords = ["weather", "temperature", "forecast", "rain", "wind", "humidity", "hot", "cold"]
    if any(k in q for k in weather_keywords):
        return "weather"

    return "rag"

# Routing proof (grader-friendly)
test_queries = [
    "What's the weather in Haifa?",
    "What is the price of bitcoin?",
    "Explain BM25 algorithm"
]
print("Routing proof:")
for tq in test_queries:
    print(f"- Query: {tq}")
    print(f"  Route chosen: {route_task(tq)}")
print("✓ Routing logic working")


Routing proof:
- Query: What's the weather in Haifa?
  Route chosen: weather
- Query: What is the price of bitcoin?
  Route chosen: crypto
- Query: Explain BM25 algorithm
  Route chosen: rag
✓ Routing logic working


In [53]:
# =========================
# Cell 9: Main agentic_answer (routes to tools)
# =========================
def agentic_answer(user_query: str) -> Dict[str, Any]:
    decision = route_task(user_query)

    if decision == "weather":
        city = extract_city(user_query)
        weather = get_weather_open_meteo(city)
        return {
            "tool_used": "weather_api",
            "decision": f"Weather keywords detected → Open-Meteo API for {city}",
            "result": weather
        }

    if decision == "crypto":
        coin = extract_crypto(user_query)
        crypto = get_crypto_price(coin)
        return {
            "tool_used": "crypto_api",
            "decision": f"Crypto keywords detected → CoinGecko API for {coin}",
            "result": crypto
        }

    rag = rag_answer(user_query, k=3)
    return {
        "tool_used": "rag",
        "decision": "Default route → RAG retrieval + Groq answer grounded in sources",
        "result": rag
    }

print("✓ agentic_answer ready")


✓ agentic_answer ready


In [54]:
# =========================
# Cell 10: Pretty display function
# =========================
def show_agentic_result(res: Dict[str, Any]) -> None:
    print("╔" + "═" * 62 + "╗")
    print(f"║ TOOL USED: {res['tool_used'].upper():49} ║")
    print("╠" + "═" * 62 + "╣")

    decision = res["decision"]
    lines = [decision[i:i+52] for i in range(0, len(decision), 52)]
    print(f"║ DECISION: {lines[0]:52} ║")
    for extra in lines[1:]:
        print(f"║           {extra:52} ║")
    print("╚" + "═" * 62 + "╝\n")

    if res["tool_used"] == "weather_api":
        w = res["result"]
        if "error" in w:
            print("❌", w["error"])
            return
        print(f"🌤️  Weather in {w['city']}, {w['country']}")
        print(f"   📍 {w['lat']:.2f}, {w['lon']:.2f}")
        print(f"   🌡️  {w['temperature_c']} °C")
        print(f"   💨 {w['wind_kmh']} km/h")
        print(f"   🕐 {w['time']}")
        print(f"   📡 {w['source']}")

    elif res["tool_used"] == "crypto_api":
        c = res["result"]
        if "error" in c:
            print("❌", c["error"])
            return
        emoji = "📈" if (c.get("change_24h") or 0) > 0 else "📉"
        print(f"💰 {c['symbol'].upper()}")
        print(f"   💵 ${c['price_usd']:,.2f}")
        print(f"   {emoji} {c['change_24h']:.2f}% (24h)")
        print(f"   📡 {c['source']}")

    else:
        rag = res["result"]
        print("📚 RAG Answer:\n" + "─" * 60)
        print(rag["answer"])
        print("\n" + "─" * 60)
        print("📖 Sources:")
        for s in rag["sources"]:
            snippet = s if len(s) <= 350 else s[:350] + "..."
            print("\n" + snippet)

print("✓ show_agentic_result ready")


✓ show_agentic_result ready


In [55]:
# =========================
# Cell 11: FINAL DEMO (shows all 3 tools)
# =========================
print("\n" + "🎯" * 25)
print("AGENTIC RAG SYSTEM - FINAL DEMO")
print("🎯" * 25)

print("\n" + "="*70)
print("DEMO 1: WEATHER API")
print("="*70)
show_agentic_result(agentic_answer("What's the weather in Tel Aviv?"))

print("\n" + "="*70)
print("DEMO 2: CRYPTO API (BONUS)")
print("="*70)
show_agentic_result(agentic_answer("What is the current price of Ethereum?"))

print("\n" + "="*70)
print("DEMO 3: RAG (IR)")
print("="*70)
show_agentic_result(agentic_answer("Explain the BM25 ranking algorithm and how it works."))

print("\n✅ All demos completed.")



🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯
AGENTIC RAG SYSTEM - FINAL DEMO
🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯🎯

DEMO 1: WEATHER API
╔══════════════════════════════════════════════════════════════╗
║ TOOL USED: WEATHER_API                                       ║
╠══════════════════════════════════════════════════════════════╣
║ DECISION: Weather keywords detected → Open-Meteo API for Tel A ║
║           viv                                                  ║
╚══════════════════════════════════════════════════════════════╝

🌤️  Weather in Tel Aviv, Israel
   📍 32.08, 34.78
   🌡️  19.5 °C
   💨 5.2 km/h
   🕐 2026-01-31T21:30
   📡 open-meteo

DEMO 2: CRYPTO API (BONUS)
╔══════════════════════════════════════════════════════════════╗
║ TOOL USED: CRYPTO_API                                        ║
╠══════════════════════════════════════════════════════════════╣
║ DECISION: Crypto keywords detected → CoinGecko API for ethereu ║
║           m                                                    ║
╚════════════════════════

In [56]:
# =========================
# Cell 12: Interactive demo (optional)
# =========================
def interactive_demo():
    """
    Interactive loop for testing queries.
    Type 'exit' to quit.
    """
    print("\n🤖 Interactive Agentic RAG Demo")
    print("=" * 60)
    print("Try queries like:")
    print("  - 'What's the weather in London?'")
    print("  - 'What is the price of bitcoin?'")
    print("  - 'Explain inverted index'")
    print("Type 'exit' to quit")
    print("=" * 60 + "\n")

    while True:
        try:
            query = input("\n💬 Your query: ").strip()

            if query.lower() in ['exit', 'quit', 'q']:
                print("👋 Goodbye!")
                break

            if not query:
                continue

            print()
            result = agentic_answer(query)
            show_agentic_result(result)

        except KeyboardInterrupt:
            print("\n👋 Goodbye!")
            break
        except Exception as e:
            print(f"❌ Error: {e}")

# Uncomment to run interactive demo:
# interactive_demo()

print("✓ Interactive demo ready (uncomment to use)")


✓ Interactive demo ready (uncomment to use)


In [ ]:
# =========================
# Cell 13: Summary stats
# =========================
avg_size = sum(len(c.page_content) for c in chunks) // max(1, len(chunks))

print("\n" + "📊" * 25)
print("SYSTEM SUMMARY")
print("📊" * 25)
print(f"📁 Documents loaded: {len(docs)}")
print(f"📄 Chunks created: {len(chunks)}")
print(f"🔍 Average chunk size: {avg_size} chars")
print("💾 Vector store: FAISS (local)")
print("📡 Embeddings: sentence-transformers/all-MiniLM-L6-v2")
print("🤖 LLM: Groq (llama-3.1-8b-instant)")
print("\n🛠️ Tools:")
print("  1) RAG")
print("  2) Weather API (Open-Meteo)")
print("  3) Crypto API (CoinGecko) ⭐ BONUS")
print("\n✅ Status: READY")
print("📊" * 25)


✓ Interactive demo ready (uncomment to use)


In [58]:
# Cell 13-SYSTEM SUMMARY
avg_size = sum(len(c.page_content) for c in chunks) // max(1, len(chunks))

print("\n" + "📊" * 25)
print("SYSTEM SUMMARY")
print("📊" * 25)
print(f"📁 Documents loaded: {len(docs)}")
print(f"📄 Chunks created: {len(chunks)}")
print(f"🔍 Average chunk size: {avg_size} chars")
print("💾 Vector store: FAISS (local)")
print("📡 Embeddings: sentence-transformers/all-MiniLM-L6-v2")
print("🤖 LLM: Groq (llama-3.1-8b-instant)")
print("\n🛠️ Tools:")
print("  1) RAG")
print("  2) Weather API (Open-Meteo)")
print("  3) Crypto API (CoinGecko) ⭐ BONUS")
print("\n✅ Status: READY")
print("📊" * 25)



📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊
SYSTEM SUMMARY
📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊
📁 Documents loaded: 1
📄 Chunks created: 5
🔍 Average chunk size: 693 chars
💾 Vector store: FAISS (local)
📡 Embeddings: sentence-transformers/all-MiniLM-L6-v2
🤖 LLM: Groq (llama-3.1-8b-instant)

🛠️ Tools:
  1) RAG
  2) Weather API (Open-Meteo)
  3) Crypto API (CoinGecko) ⭐ BONUS

✅ Status: READY
📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊📊
